In [ ]:
import numpy as np

from pyxpcs.config import PyXPCSDict

from pyxpcs.utils import config_from_8idhdf5, load_results
from pyxpcs.viz import plot_pix_sum, plot_as_image



In [ ]:
Rigaku_FILE = ""
HDF5_FILE = ""

In [ ]:
config = config_from_8idhdf5(HDF5_FILE)
try:
    import pandas as pd
    df = pd.DataFrame.from_dict(config, orient='index')
    print(df)
except:
    pass

In [ ]:
with open(Rigaku_FILE) as f:
    a = np.fromfile(f, dtype=np.uint64)
    
    b = (a >> 5+11)
    pix_ind = (b & 2**21-1).astype(int)
    pix_count = (a & 2**12-1).astype(int)
    pix_frame = (a >> 64-24).astype(int)


In [ ]:
pix_ind

In [ ]:
max(pix_frame)+1


In [ ]:
pix_count

In [ ]:
%%time
import libpyxpcs as libpx


In [ ]:
%%time
sparse_data = libpx.sparse_lil_index_value_count(pix_ind, pix_frame, pix_count, np.max(pix_ind)+1)

In [ ]:
%%time
mini_config = {
    'frames': config['frames'] if 'frames' in config else max(pix_frame)+1,
    'pixels': config['cols'] * config['rows'],
    'delays_per_level': config['delays_per_level'] if 'delays_per_level' in config else 4,
    'rows': config['rows'],
    'cols': config['cols'],  
}

G2, IP, IF = libpx.multitau(sparse_data, mini_config)

In [ ]:
G2_plot = G2[0].reshape(config['rows'], config['cols'])
plot_as_image(G2_plot)

In [ ]:
G2.shape